In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install fastai=="1.0.61"
# !pip install torchsummary

     |████████████████████████████████| 239 kB 4.0 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19191 sha256=4b45df48b3f378928c801fb0eb4b8f210dbe2e368588a541008160b2d9875714
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
Successfully built nvidia-ml-py3
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.5.3
    Uninstalling fastai-2.5.3:
      Successfully uninstalled fastai-2.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch
# from google.colab import drive
# drive.mount('/content/drive')
from glob import glob
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# from torchsummary import summary
from fastai.metrics import accuracy
from fastai.vision import *
import numpy as np
!pip install fft-conv-pytorch
from fft_conv_pytorch import fft_conv, FFTConv2d

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# from fastai.vision.data import normalize
path = "../input/covid19grayscaleds/covid-19_dataset_grayscale"
np.random.seed(41)
defaults = SimpleNamespace(cmap='L', return_fig=False, silent=False)
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(256,256), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [7]:
print (data.one_batch()[0].shape)
print (data.classes, data.c, len(data.train_ds), len(data.valid_ds))
print (data.train_ds.classes)
print (data.valid_ds.classes)

torch.Size([16, 1, 256, 256])
['covid-19', 'no_findings', 'pneumonia'] 3 900 225
['covid-19', 'no_findings', 'pneumonia']
['covid-19', 'no_findings', 'pneumonia']


With FFT Conv Layers

In [8]:
def fft_conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        FFTConv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )
def fft_triple_conv(ni, nf,size):
    return nn.Sequential(
        fft_conv_block(ni, nf,size),
        conv_block(nf, ni, size=1),  
        fft_conv_block(ni, nf, size)
    )
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
fft_model = nn.Sequential(
    fft_conv_block(1, 8, 21),
    maxpooling(),
    fft_conv_block(8, 16, 17),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,3),
    nn.Flatten(),
    nn.Linear(507, 3)
)

In [ ]:
fft_learn = Learner(data, fft_model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn.summary())

In [ ]:
fft_learn.fit_one_cycle(100, max_lr=4e-3)

In [ ]:
probs,targets = fft_learn.get_preds(ds_type=DatasetType.Valid) 
accuracy(probs,targets)
fft_learn.recorder.plot_losses()

Model 2

In [ ]:
fft_model2 = nn.Sequential(
    fft_conv_block(1, 8, 21),
    maxpooling(),
    fft_conv_block(8, 16, 17),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,3),
    nn.Flatten(),
    nn.Linear(507, 3)
)
fft_learn2 = Learner(data, fft_model2, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn2.summary())
fft_learn2.fit_one_cycle(100, max_lr=8e-4)
probs,targets = fft_learn2.get_preds(ds_type=DatasetType.Valid) 
accuracy(probs,targets)
fft_learn2.recorder.plot_losses()

Model 3

In [11]:
# data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(512,512), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"
fft_model3 = nn.Sequential(
    fft_conv_block(1,8,25),
    maxpooling(),
    fft_conv_block(8,16,25),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,3),
    nn.Flatten(),
    nn.Linear(1323, 3)
)
fft_learn3 = Learner(data, fft_model3, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn3.summary())
fft_learn3.fit_one_cycle(100, max_lr=5e-3)
probs,targets = fft_learn3.get_preds(ds_type=DatasetType.Valid) 
accuracy(probs,targets)
# fft_learn3.recorder.plot_losses()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
_FFTConv             [8, 512, 512]        5,000      True      
______________________________________________________________________
BatchNorm2d          [8, 512, 512]        16         True      
______________________________________________________________________
LeakyReLU            [8, 512, 512]        0          False     
______________________________________________________________________
MaxPool2d            [8, 256, 256]        0          False     
______________________________________________________________________
_FFTConv             [16, 256, 256]       80,000     True      
______________________________________________________________________
BatchNorm2d          [16, 256, 256]       32         True      
______________________________________________________________________
LeakyReLU            [16, 256, 256]       0          False     
___________________________________________________

epoch,train_loss,valid_loss,accuracy,time
0,0.817022,0.565847,0.764444,00:35
1,0.725286,0.493367,0.813333,00:35
2,0.657558,0.497793,0.777778,00:35
3,0.622079,0.450680,0.813333,00:34
4,0.628701,0.642694,0.702222,00:36
5,0.634365,0.897673,0.600000,00:34
6,0.627412,0.570187,0.760000,00:34
7,0.661223,0.539620,0.764444,00:34
8,0.613639,0.425722,0.817778,00:35
9,0.645591,0.879508,0.631111,00:34


tensor(0.8267)

In [16]:
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(256,256), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"
fft_model4 = nn.Sequential(
    conv_block(1,8),
    maxpooling(),
    conv_block(8,16),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,3),
    nn.Flatten(),
    nn.Linear(507, 3)
)
fft_learn4 = Learner(data, fft_model4, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn4.summary())
fft_learn4.fit_one_cycle(100, max_lr=5e-3)
probs,targets = fft_learn4.get_preds(ds_type=DatasetType.Valid) 
accuracy(probs,targets)

Sequential
Layer (type)         Output Shape         Param #    Trainable 
_FFTConv             [8, 256, 256]        3,528      True      
______________________________________________________________________
BatchNorm2d          [8, 256, 256]        16         True      
______________________________________________________________________
LeakyReLU            [8, 256, 256]        0          False     
______________________________________________________________________
MaxPool2d            [8, 128, 128]        0          False     
______________________________________________________________________
_FFTConv             [16, 128, 128]       36,992     True      
______________________________________________________________________
BatchNorm2d          [16, 128, 128]       32         True      
______________________________________________________________________
LeakyReLU            [16, 128, 128]       0          False     
___________________________________________________

epoch,train_loss,valid_loss,accuracy,time
0,0.804243,0.734150,0.680000,00:23
1,0.714924,0.573794,0.746667,00:22
2,0.685435,0.618540,0.706667,00:23
3,0.640070,0.618319,0.746667,00:22
4,0.621064,0.793769,0.662222,00:23
5,0.629019,0.599595,0.737778,00:22
6,0.604919,0.713611,0.742222,00:22
7,0.625657,0.615022,0.746667,00:22
8,0.645796,0.554397,0.800000,00:23
9,0.639510,0.770991,0.671111,00:22


tensor(0.8267)

In [ ]:
fft_learn4.recorder.plot_losses()

In [24]:
vgg_model = nn.Sequential(
    
    nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),

    nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2), 
    
    # nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.MaxPool2d(kernel_size=2, stride=2), 
    
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Flatten(),
    nn.Linear(131072, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    # nn.Linear(4096, 4096),
    # nn.ReLU(inplace=True),
    # nn.Dropout(0.5),
    nn.Linear(512, 3)
)


In [25]:
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(256,256), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"
vgg_learn = Learner(data, vgg_model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(vgg_learn.summary())

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [16, 256, 256]       160        True      
______________________________________________________________________
ReLU                 [16, 256, 256]       0          False     
______________________________________________________________________
Conv2d               [16, 256, 256]       2,320      True      
______________________________________________________________________
ReLU                 [16, 256, 256]       0          False     
______________________________________________________________________
MaxPool2d            [16, 128, 128]       0          False     
______________________________________________________________________
Conv2d               [32, 128, 128]       4,640      True      
______________________________________________________________________
ReLU                 [32, 128, 128]       0          False     
___________________________________________________

In [26]:
import time
start = time.time()
vgg_learn.fit_one_cycle(100, max_lr=4e-3)
end = time.time()
print(end-start)

epoch,train_loss,valid_loss,accuracy,time
0,1.001638,0.946729,0.431111,00:23
1,0.991617,0.935088,0.480000,00:22
2,0.989380,0.956835,0.480000,00:22
3,0.969459,0.834807,0.595556,00:22
4,0.838718,0.672648,0.684444,00:23
5,0.765000,0.647176,0.688889,00:22
6,0.738895,0.628361,0.702222,00:22
7,0.726907,0.651437,0.728889,00:23
8,0.729611,0.672681,0.688889,00:22
9,0.720174,0.705872,0.684444,00:22


2289.208954811096
